In [0]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as pltb
from scipy.special import comb,factorial

def sYlm(l,m,s,th,p):
  temp = 0
  for r in range(0,l-s+1):
    temp += comb(l-s,r,exact = True)*comb(l+s,r+s-m,exact = True)*(-1**(l-r-s))/(np.tan(th/2)**(2*r + s - m) )
  
  prefact = (-1**m)*np.sqrt(factorial(l+m)*(2*l + 1)/(4*np.pi*factorial(l+s)*factorial(l-s)))*(np.sin(th/2)**(2*l))
  
  return temp*prefact*np.exp(m*p*1J)

# Generating Waveforms from the Backwards One Body Model

### Author: Siddharth Mahesh

## Waveform Characterisation in the BOB model

In the Backwards-One-Body model, the late inspiral and the merger-ringdown parts of the binary black hole merger are assumed to dominate the waveform. At these stages of the merger, the final black hole is the relavent background spacetime, the waveform amplitude and phase are given by the evolution of the Quasi-Normal Modes (QNMs) of perturbations about the final black hole.

### Amplitude

The Amplitude of the gravitational waveform is first recorded in terms of the amplitude of the Weyl Scalar as:

$$
|\psi_4| = A_p\text{sech}\left[\gamma(t - t_p)\right]
$$

Where, $t_p$ corresponds to the time at which the waveform reaches peak amplitude. The strain amplitude is then given by:

$$
|h_{lm}|  = \frac{1}{\omega_{lm}^2}|\psi_4|
$$

Here, $\omega$ and $\gamma$ in the previous equation denote the orbital frequency of the null congruence as well as the corresponding Lyapunov Exponent, respectively. The Lyapunov exponent is given by:

$$
\gamma = \sqrt{3\omega^2\left(1 - \frac{2M}{r_{lr}} + \frac{a^2}{r_{lr}^2}\right)}
$$

Where, $(M,a)$ are the mass and spin of the final Kerr black hole and $r_{lr}$ is the location of the light ring given by:

$$
r_{lr} = 2M\left(1 + \cos\left(\frac{2}{3}\cos^{-1}\left(-\frac{a}{M}\right)\right)\right)
$$

And the frequency at the light ring is given by:

$$
\omega = \frac{M^{1/2}}{r^{3/2} + aM^{1/2}}
$$

In [0]:
M , a = sp.symbols("M a")
A_p , t_p , t = sp.symbols("A_p t t_p")

r_lr = 2*M*(1+sp.cos(sp.Rational(2,3)*sp.acos(-a/M)))
w_lr = sp.sqrt(M)/(sp.sqrt(r_lr**3) + a*sp.sqrt(M))
gamma = sp.sqrt(3*(w_lr**2)*(1 - 2*M/r_lr + a**2/r_lr**2))

psi4 = A_p / sp.cosh(gamma * (t - t_p))

print(psi4)

A_p/cosh(sqrt(3)*sqrt((32*M**2*cos(acos(-a/M)/3)**6 - 16*M**2*cos(acos(-a/M)/3)**4 + a**2*(cos(2*acos(-a/M)/3) + 1))/(M*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**2*(cos(2*acos(-a/M)/3) + 1)*cos(acos(-a/M)/3)**4))*(t - t_p)/4)


### Phase 

The phase of the waveform is given by:

$$
\mathbf{\Phi} = \text{arctan}_+ + \text{arctanh}_+ - \text{arctan}_- - \text{arctan}_-
$$

Where the individual terms:

$$
\text{arctan[h]}_\pm = \kappa_\pm\tau\left[\text{arctan[h]}\left(\frac{\Omega}{\kappa_\pm}\right) - \text{arctan[h]}\left(\frac{\Omega_0}{\kappa_\pm}\right)\right]
$$

The term $\kappa$ is then given by:

$$
\kappa_\pm = \left\{\Omega_0^4 \pm k\left[1 - \mp \tanh\left(\frac{t_0 - t_p}{\tau}\right)\right]\right\}
$$

And $\Omega$ given by:

$$
\Omega = \left\{\Omega_0^4 + k\left[\tanh\left(\frac{t - t_p}{\tau} \right) - \tanh\left(\frac{t_0 - t_p}{\tau} \right)\right]\right\}^{1/4}
$$

Where, $k$ is given by:

$$
k = \left(\frac{\Omega^4_{\text{QNM}} - \Omega^4_0}{1 - \tanh\left\{(t_0 - t_p)/\tau\right\}}\right)
$$

Where, $t_0$ , $\Omega_0$ correspond to the time and corresponding angular frequency of the perturber when it transitions from inspiral. $\Omega_{\text{QNM}} = \omega_{\text{QNM}}/m$ is the orbital frequency associated with the light ring perturbation which sources quasinormal mode emission at frequency $\omega_{\text{QNM}}$. Lastly, $\tau = \gamma^{-1}$ is the damping timescale related to the Lyapunov exponent.

In [0]:
W_QNM = w_lr/2
W_0 , t_0 = sp.symbols("W_0 t_0")

k = (W_QNM**4 - W_0**4)/(1 - sp.tanh(gamma*(t_0 - t_p)))
W = (W_0**4 + k*(sp.tanh(gamma*(t - t_p)) - sp.tanh(gamma*(t_0 - t_p))))**(1/4)

kappa_p = W_0**4 + k * ( 1 + sp.tanh(gamma*(t_0 - t_p)))
kappa_m = W_0**4 - k * ( 1 - sp.tanh(gamma*(t_0 - t_p)))

arctan_p = kappa_p*(sp.atan(W/kappa_p) - sp.atan(W_0/kappa_p))/gamma
arctan_m = kappa_m*(sp.atan(W/kappa_m) - sp.atan(W_0/kappa_m))/gamma
arctan_hp = kappa_p*(sp.atanh(W/kappa_p) - sp.atanh(W_0/kappa_p))/gamma
arctan_hm = kappa_m*(sp.atanh(W/kappa_m) - sp.atanh(W_0/kappa_m))/gamma

Phi = arctan_p + arctan_hp - arctan_m - arctan_hm

w_lm = 2*W

print(Phi)

-2*sqrt(3)*(W_0**4 - (M**2/16 - W_0**4*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**4)*(-tanh(sqrt(3)*sqrt((4*M - 4*M/(cos(2*acos(-a/M)/3) + 1) + a**2/(4*M*cos(acos(-a/M)/3)**4))/(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**2)*(-t + t_0)/2) + 1)/((sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**4*(tanh(sqrt(3)*sqrt((32*M**2*cos(acos(-a/M)/3)**6 - 16*M**2*cos(acos(-a/M)/3)**4 + a**2*(cos(2*acos(-a/M)/3) + 1))/(M*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**2*(cos(2*acos(-a/M)/3) + 1)*cos(acos(-a/M)/3)**4))*(t - t_0)/4) + 1)))*(-atan(W_0/(W_0**4 - (M**2/16 - W_0**4*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**4)*(-tanh(sqrt(3)*sqrt((4*M - 4*M/(cos(2*acos(-a/M)/3) + 1) + a**2/(4*M*cos(acos(-a/M)/3)**4))/(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**2)*(-t + t_0)/2) + 1)/((sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**4*(tanh(sqrt(3)*sqrt((32*M**2*cos(acos(-a/M)/3)**6 - 16*M**2*cos(acos(-a/M)/3)**4 + a**2*(cos(2*acos(-a/M)/3) + 1))/(M*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(

### The Waveform

The waveform can be characterised by the complex sum that encodes the strain of the gravitational waves observed in both polarisations.

$$
h = h_+ + ih_\times
$$

This complex strain is then subject to a spin weighted spherical harmonic decomposition:

$$
h = \sum\limits_{l=2}^{\infty}\sum\limits_{m = -l}^{l} h_{lm}(t,R) _{-2}Y^m_l(\theta,\phi) 
$$

Where , $(R,\theta,\phi)$ represent the radius and polar angles at which the strain is extracted.

Each harmonic mode $h_{lm}$ is a propagating wave of the form:

$$
h_{lm} = |h_{lm}|e^{i\mathbf{m\Phi_{lm}}}
$$


In [0]:
modh_lm = psi4/(w_lm**2)

h_lm = modh_lm*sp.exp((1J)*2*Phi)
print(h_lm)

1.0*A_p*((16*W_0**4*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**4*(tanh(sqrt(3)*sqrt((32*M**2*cos(acos(-a/M)/3)**6 - 16*M**2*cos(acos(-a/M)/3)**4 + a**2*(cos(2*acos(-a/M)/3) + 1))/(M*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**2*(cos(2*acos(-a/M)/3) + 1)*cos(acos(-a/M)/3)**4))*(t - t_0)/4) + 1) + (M**2 - 16*W_0**4*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**4)*(tanh(sqrt(3)*sqrt((32*M**2*cos(acos(-a/M)/3)**6 - 16*M**2*cos(acos(-a/M)/3)**4 + a**2*(cos(2*acos(-a/M)/3) + 1))/(M*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**2*(cos(2*acos(-a/M)/3) + 1)*cos(acos(-a/M)/3)**4))*(t - t_0)/4) - tanh(sqrt(3)*sqrt((32*M**2*cos(acos(-a/M)/3)**6 - 16*M**2*cos(acos(-a/M)/3)**4 + a**2*(cos(2*acos(-a/M)/3) + 1))/(M*(sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**2*(cos(2*acos(-a/M)/3) + 1)*cos(acos(-a/M)/3)**4))*(t - t_p)/4)))/((sqrt(M)*a + 8*sqrt(M**3*cos(acos(-a/M)/3)**6))**4*(tanh(sqrt(3)*sqrt((32*M**2*cos(acos(-a/M)/3)**6 - 16*M**2*cos(acos(-a/M)/3)**4 + a**2*(cos(2*acos(-a/M)/3)

## Computing the Peak Amplitude Time

In order to generate waveforms, it is required to enforce continuity with the waveform from the inspiral at some transition time $t_0$.

In the EOB formalism, the light ringis associated with the time of peak strain and therefore, for determination of the appropriate time $t_0$, it is first crucial to compute the $t^{(h)}_p$ at which the strain is at the maximum. To do this, we extremise the strain

$$
\dot{h}_{lm} = 0
$$

## Matching the Waveform to the EOB Inspiral

